### Introduction

VentBuddy was developed to provide users with a safe and supportive space to express their thoughts and emotions without judgment. The chatbot takes user input in the form of free text and analyzes it using a trained classification model to identify the underlying emotional category. Based on the predicted category, VentBuddy generates an empathetic and supportive response tailored to the user’s emotional state. The system is designed to encourage emotional expression while clearly avoiding medical advice or diagnosis.

In [1]:
!pip install streamlit pyngrok scikit-learn nltk textblob torch -q

print(" All packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.4 MB/s eta 0:00:00
 All packages installed successfully!


In [2]:
import nltk

print(" Downloading NLTK data...")
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt', quiet=True)

print(" NLTK setup complete!")

 NLTK setup complete!


In [12]:
from google.colab import drive
import pickle
import joblib

# Mount Drive
print(" Mounting Google Drive...")
drive.mount('/content/drive')
print(" Drive mounted!\n")

model_path = '/content/drive/MyDrive/Data602_Project/Models/sbert_logreg_model.pkl'

print(f" Loading model from: {model_path}")

try:
    print(" Attempting pickle load...")
    with open(model_path, 'rb') as f:
        checkpoint = pickle.load(f, encoding='bytes')

    print(" Model loaded successfully with pickle!")

except Exception as e1:
    print(f"  Pickle failed: {e1}")

    try:
        print("\n Trying joblib...")
        checkpoint = joblib.load(model_path)
        print(" Model loaded successfully with joblib!")

    except Exception as e2:
        print(f" Joblib failed: {e2}")

        try:
            # Method 3: Try pickle with latin1 encoding
            print("\n Trying pickle with latin1...")
            with open(model_path, 'rb') as f:
                checkpoint = pickle.load(f, encoding='latin1')
            print(" Model loaded successfully with latin1!")

        except Exception as e3:
            print(f"\n All methods failed:")
            print(f"   Pickle: {e1}")
            print(f"   Joblib: {e2}")
            print(f"   Latin1: {e3}")
            raise

print(f"\n Model structure:")
print(f"   Type: {type(checkpoint)}")

if isinstance(checkpoint, dict):
    print(f"   Keys: {list(checkpoint.keys())}")


    for key, value in checkpoint.items():
        print(f"   - {key}: {type(value)}")


        if key == 'model' and hasattr(value, '__class__'):
            print(f"     Model type: {value.__class__.__name__}")


            if hasattr(value, 'classes_'):
                print(f"     Categories: {list(value.classes_)}")


        if key == 'vectorizer' and hasattr(value, '__class__'):
            print(f"     Vectorizer type: {value.__class__.__name__}")
else:
    print(f"   Content: {checkpoint}")

print("\n Saving for VentBuddy...")
with open('model_checkpoint.pkl', 'wb') as f:
    pickle.dump(checkpoint, f)

print(" Model ready for VentBuddy!")

# Try to extract and show categories
print("\n  Model categories:")
try:
    if isinstance(checkpoint, dict):
        if 'model' in checkpoint and hasattr(checkpoint['model'], 'classes_'):
            categories = checkpoint['model'].classes_
            print(f"   {list(categories)}")
        elif 'classes' in checkpoint:
            print(f"   {checkpoint['classes']}")
        elif 'label_encoder' in checkpoint:
            print(f"   {checkpoint['label_encoder'].classes_}")
    else:
        if hasattr(checkpoint, 'classes_'):
            print(f"   {list(checkpoint.classes_)}")
except Exception as e:
    print(f"   Could not extract categories: {e}")

print("\n All done! Ready to create VentBuddy.")

 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Drive mounted!

 Loading model from: /content/drive/MyDrive/Data602_Project/Models/sbert_logreg_model.pkl
 Attempting pickle load...
  Pickle failed: invalid load key, '\x0b'.

 Trying joblib...
 Model loaded successfully with joblib!

 Model structure:
   Type: <class 'dict'>
   Keys: ['model_type', 'sbert_model', 'class_labels', 'model']
   - model_type: <class 'str'>
   - sbert_model: <class 'str'>
   - class_labels: <class 'list'>
   - model: <class 'sklearn.linear_model._logistic.LogisticRegression'>
     Model type: LogisticRegression
     Categories: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]

 Saving for VentBuddy...
 Model ready for VentBuddy!

  Model categories:
   [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]

 All done! Ready to create VentBuddy.


In [10]:
%%writefile app.py
import streamlit as st
import pickle
import torch
import re
from datetime import datetime
import random

# MODEL LOADING
@st.cache_resource
def load_model():
    """Load your trained PyTorch model"""
    try:
        with open('model_checkpoint.pkl', 'rb') as f:
            checkpoint = pickle.load(f)

        model = checkpoint.get('model_state_dict')
        vectorizer = checkpoint.get('vectorizer')
        label_encoder = checkpoint.get('label_encoder')

        return checkpoint, vectorizer, label_encoder
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None, None, None

# TEXT PREPROCESSING
def preprocess_text(text):
    """Clean text for model input"""
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join(text.split())
    return text

# PREDICTION FUNCTION (RETURNS ALL PROBABILITIES)
def predict_category(text, checkpoint, vectorizer, label_encoder):
    """Make prediction and return ALL category probabilities"""
    try:
        cleaned_text = preprocess_text(text)

        # Categories
        categories = ['anxiety', 'depression', 'suicidewatch', 'lonely', 'mentalhealth']

        if vectorizer is not None:
            text_vector = vectorizer.transform([cleaned_text])

            # If using sklearn model with predict_proba
            if hasattr(checkpoint.get('model'), 'predict_proba'):
                probabilities = checkpoint['model'].predict_proba(text_vector)[0]
                prediction = checkpoint['model'].predict(text_vector)[0]

                # Create probability dictionary
                prob_dict = {}
                for i, cat in enumerate(categories):
                    prob_dict[cat] = probabilities[i] * 100

                return prediction, prob_dict
            else:
                # Fallback
                prediction, confidence = simple_classification(text)
                prob_dict = {cat: 0.0 for cat in categories}
                prob_dict[prediction] = confidence
                return prediction, prob_dict
        else:
            prediction, confidence = simple_classification(text)
            prob_dict = {cat: 0.0 for cat in categories}
            prob_dict[prediction] = confidence
            return prediction, prob_dict

    except Exception as e:
        st.error(f"Prediction error: {e}")
        prob_dict = {cat: 20.0 for cat in categories}
        return 'mentalhealth', prob_dict

# SIMPLE KEYWORD-BASED CLASSIFICATION
def simple_classification(text):
    """Simple rule-based classification"""
    text_lower = text.lower()

    if any(word in text_lower for word in ['suicide', 'suicidal', 'kill myself', 'end it all', 'want to die']):
        return 'suicidewatch', 90.0
    elif any(word in text_lower for word in ['anxious', 'anxiety', 'worried', 'panic', 'nervous', 'stress']):
        return 'anxiety', 80.0
    elif any(word in text_lower for word in ['depressed', 'depression', 'sad', 'hopeless', 'empty', 'numb']):
        return 'depression', 80.0
    elif any(word in text_lower for word in ['alone', 'lonely', 'isolated', 'nobody', 'no friends']):
        return 'lonely', 80.0
    else:
        return 'mentalhealth', 70.0

# FRIENDLY RESPONSES
def get_buddy_response(category, prob_dict):
    """Generate short, friendly responses"""

    responses = {
        'anxiety': {
            'messages': [
                "I can tell you're feeling anxious about this. That sounds really tough.",
                "Hey, I hear you. Anxiety can feel so overwhelming, especially when it hits hard.",
                "That sounds stressful. I get it - when anxiety kicks in, everything feels bigger.",
                "I'm sorry you're feeling this way. Anxiety is exhausting, isn't it?"
            ],
            'tips': [
                "Try taking a few deep breaths - in for 4, out for 4. It really helps calm things down.",
                "Maybe step outside for a minute? Fresh air can help reset your mind.",
                "Put your hand on your chest and feel your heartbeat. You're here, you're okay.",
                "Try naming 5 things you can see around you. It helps ground you in the present."
            ]
        },
        'depression': {
            'messages': [
                "I'm really sorry you're feeling this way. That sounds incredibly hard.",
                "I hear you. Depression makes everything feel heavier, doesn't it?",
                "That must feel so exhausting. I'm glad you're sharing this with me.",
                "I can sense how difficult this is for you right now. You're not alone in this."
            ],
            'tips': [
                "Even if it's just brushing your teeth or making your bed - small steps count.",
                "Try to get outside for just 5 minutes today. Sunlight helps more than you'd think.",
                "Text someone you trust, even just 'hey'. Connection helps, even when it feels impossible.",
                "Be gentle with yourself today. You're doing the best you can."
            ]
        },
        'suicidewatch': {
            'messages': [
                "I'm really concerned about what you're going through. Please know you're not alone.",
                "What you're feeling sounds incredibly painful. Please reach out for help - you deserve support.",
                "I'm worried about you. These feelings are serious, and you don't have to face them alone.",
                "Thank you for sharing this with me. Please talk to someone who can help you through this."
            ],
            'tips': [
                "Please call 988 (Suicide & Crisis Lifeline) or text HOME to 741741. They're there 24/7.",
                "Can you reach out to someone you trust right now? You don't have to go through this alone."
            ]
        },
        'lonely': {
            'messages': [
                "Loneliness is so hard. I'm sorry you're feeling this way.",
                "I hear you. Feeling lonely can be really painful, even when you're around people.",
                "That sounds really isolating. Thank you for opening up about this.",
                "I can sense how much you're hurting. Loneliness is one of the hardest feelings."
            ],
            'tips': [
                "Try reaching out to someone, even with a simple 'hey, how are you?' It's a start.",
                "Join an online community about something you like - connection can start small.",
                "Sometimes just being around people helps. Coffee shop, library, anywhere with people nearby.",
                "You're not alone in feeling this way. Many people understand what you're going through."
            ]
        },
        'mentalhealth': {
            'messages': [
                "Thank you for sharing what's going on. I'm here to listen.",
                "I hear you. What you're going through sounds really challenging.",
                "That sounds like a lot to handle. I'm glad you're talking about it.",
                "I can tell this is weighing on you. You're not alone in dealing with this."
            ],
            'tips': [
                "Taking care of your mental health is just as important as physical health.",
                "Small steps are still steps forward. Be patient with yourself.",
                "Consider talking to someone - a friend, family member, or counselor. It helps.",
                "Give yourself permission to not be okay sometimes. That's part of being human."
            ]
        }
    }

    category_lower = category.lower()
    response_set = responses.get(category_lower, responses['mentalhealth'])

    message = random.choice(response_set['messages'])
    tip = random.choice(response_set['tips'])

    return message, tip

# PAGE CONFIGURATION
st.set_page_config(
    page_title="VentBuddy",
    page_icon="💙",
    layout="centered",
    initial_sidebar_state="collapsed"
)

# CUSTOM CSS
st.markdown("""
<style>
    /* Hide sidebar completely */
    [data-testid="stSidebar"] {
        display: none;
    }

    .main {
        background-color: #f5f7fa;
        max-width: 900px;
    }

    .chat-message {
        padding: 15px;
        border-radius: 15px;
        margin: 10px 0;
        animation: fadeIn 0.3s;
    }

    .user-message {
        background-color: #667eea;
        color: white;
        margin-left: 20%;
        text-align: left;
    }

    .bot-message {
        background-color: #ffffff;
        color: #333;
        margin-right: 20%;
        border: 1px solid #e0e0e0;
    }

    .tip-box {
        background-color: #e3f2fd;
        padding: 12px;
        border-radius: 10px;
        margin: 10px 0;
        border-left: 4px solid #2196F3;
        font-size: 14px;
    }

    .confidence-bar {
        background-color: #f0f0f0;
        border-radius: 10px;
        padding: 8px;
        margin: 8px 0;
    }

    .confidence-fill {
        background: linear-gradient(90deg, #667eea, #764ba2);
        height: 8px;
        border-radius: 5px;
        transition: width 0.3s;
    }

    .stTextInput>div>div>input {
        border-radius: 25px;
        border: 2px solid #667eea;
        padding: 12px 20px;
        font-size: 16px;
    }

    .stButton>button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-radius: 25px;
        padding: 12px 25px;
        border: none;
        font-weight: 600;
        width: 100%;
        min-height: 45px;
    }

    .stButton>button:hover {
        transform: translateY(-2px);
        box-shadow: 0 5px 15px rgba(102, 126, 234, 0.4);
    }

    @keyframes fadeIn {
        from { opacity: 0; transform: translateY(10px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .header-container {
        text-align: center;
        padding: 30px 20px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border-radius: 15px;
        margin-bottom: 30px;
    }

    .crisis-alert {
        background-color: #ffebee;
        color: #c62828;
        padding: 15px;
        border-radius: 10px;
        border-left: 4px solid #c62828;
        margin: 15px 0;
        font-weight: 600;
    }
</style>
""", unsafe_allow_html=True)

# HEADER
st.markdown("""
<div class="header-container">
    <h1 style='margin: 0; font-size: 2em;'>💙 VentBuddy</h1>
    <p style='margin: 10px 0 0 0; font-size: 1.1em; opacity: 0.95;'>
        Your friend who's always here to listen
    </p>
</div>
""", unsafe_allow_html=True)

# LOAD MODEL
checkpoint, vectorizer, label_encoder = load_model()

# INITIALIZE SESSION STATE
if 'messages' not in st.session_state:
    st.session_state.messages = []

# DISPLAY CHAT HISTORY
chat_container = st.container()

with chat_container:
    if not st.session_state.messages:
        st.markdown("""
        <div class="bot-message chat-message">
            <p style='margin: 0;'>👋 <strong>Hey there! I'm VentBuddy.</strong></p>
            <p style='margin: 10px 0 0 0;'>I'm here to listen, no judgment. Just say what's on your mind.</p>
        </div>
        """, unsafe_allow_html=True)

    for msg in st.session_state.messages:
        if msg['role'] == 'user':
            st.markdown(f"""
            <div class="user-message chat-message">
                <p style='margin: 0;'>{msg['content']}</p>
            </div>
            """, unsafe_allow_html=True)
        else:
            # Bot message
            crisis = msg.get('crisis', False)

            if crisis:
                st.markdown("""
                <div class="crisis-alert">
                    🚨 <strong>I'm really worried about you.</strong> Please reach out for help:<br>
                    • Call/Text <strong>988</strong> (Suicide & Crisis Lifeline)<br>
                    • Text <strong>HOME</strong> to <strong>741741</strong> (Crisis Text Line)
                </div>
                """, unsafe_allow_html=True)

            st.markdown(f"""
            <div class="bot-message chat-message">
                <p style='margin: 0;'>{msg['content']}</p>
            </div>
            """, unsafe_allow_html=True)

            # Show ALL category probabilities
            if msg.get('prob_dict'):
                prob_dict = msg['prob_dict']

                # Sort by probability (highest first)
                sorted_probs = sorted(prob_dict.items(), key=lambda x: x[1], reverse=True)

                st.markdown("<div style='margin: 10px 0; padding: 10px; background: #f8f9fa; border-radius: 10px;'>", unsafe_allow_html=True)
                st.markdown("<p style='margin: 0 0 8px 0; font-weight: 600; color: #555;'>📊 Analysis Breakdown:</p>", unsafe_allow_html=True)

                for category, prob in sorted_probs:
                    if prob > 5:
                        if prob >= 50:
                            color = "#667eea"
                        elif prob >= 30:
                            color = "#9b59b6"
                        else:
                            color = "#95a5a6"

                        st.markdown(f"""
                        <div class="confidence-bar">
                            <div style='display: flex; justify-content: space-between; margin-bottom: 4px;'>
                                <span style='font-size: 13px; font-weight: 600; color: #333;'>{category.title()}</span>
                                <span style='font-size: 13px; font-weight: 600; color: {color};'>{prob:.1f}%</span>
                            </div>
                            <div class="confidence-fill" style='width: {prob}%; background: {color};'></div>
                        </div>
                        """, unsafe_allow_html=True)

                st.markdown("</div>", unsafe_allow_html=True)

            if msg.get('tip'):
                st.markdown(f"""
                <div class="tip-box">
                    💡 <strong>One thing that might help:</strong><br>
                    {msg['tip']}
                </div>
                """, unsafe_allow_html=True)

# INPUT AREA
st.markdown("<br>", unsafe_allow_html=True)

col1, col2 = st.columns([5, 1])

with col1:
    user_input = st.text_input(
        "Message",
        placeholder="Type what's on your mind...",
        label_visibility="collapsed",
        key="user_input_box"
    )

with col2:
    send_button = st.button("Send", use_container_width=True)

# PROCESS INPUT
if send_button and user_input.strip():
    st.session_state.messages.append({
        'role': 'user',
        'content': user_input
    })

    # Get prediction with ALL probabilities
    if checkpoint is not None:
        prediction, prob_dict = predict_category(user_input, checkpoint, vectorizer, label_encoder)
    else:
        prediction, confidence = simple_classification(user_input)
        categories = ['anxiety', 'depression', 'suicidewatch', 'lonely', 'mentalhealth']
        prob_dict = {cat: 0.0 for cat in categories}
        prob_dict[prediction] = confidence

    # Check for crisis
    crisis_keywords = ['suicide', 'suicidal', 'kill myself', 'end it all',
                      'want to die', 'ending it all', 'hurt myself', 'harm myself',
                      'no reason to live', 'better off dead']
    is_crisis = any(keyword in user_input.lower() for keyword in crisis_keywords)

    # Get buddy response
    message, tip = get_buddy_response(prediction, prob_dict)

    # Add bot message
    st.session_state.messages.append({
        'role': 'assistant',
        'content': message,
        'category': prediction,
        'prob_dict': prob_dict,
        'tip': tip,
        'crisis': is_crisis
    })

    st.rerun()

st.markdown("<br>", unsafe_allow_html=True)
st.markdown("""
<div style='text-align: center; color: #999; font-size: 13px; padding: 10px;'>
    💙 VentBuddy is here to listen, but not a replacement for professional help
</div>
""", unsafe_allow_html=True)

Overwriting app.py


In [11]:
import sys
import time
import subprocess

print(" Launching VentBuddy with Cloudflare Tunnel...\n")

# Kill old processes
print("🧹 Cleaning up...")
!pkill -9 -f streamlit
!pkill -9 -f cloudflared

time.sleep(3)

# Download cloudflared
print(" Setting up Cloudflare Tunnel...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null 2>&1

print("⏳ Starting Streamlit...")

# Start streamlit
get_ipython().system_raw(f'{sys.executable} -m streamlit run app.py --server.port 8501 --server.headless true &')

# Wait for server
time.sleep(15)

print(" Creating public tunnel (this takes ~20 seconds)...\n")

# Start cloudflare tunnel
process = subprocess.Popen(
    ['cloudflared', 'tunnel', '--url', 'http://localhost:8501'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Wait and capture the URL
url_found = False
for line in iter(process.stdout.readline, ''):
    print(line.strip())
    if 'trycloudflare.com' in line:
        # Extract URL
        import re
        match = re.search(r'https://[^\s]+\.trycloudflare\.com', line)
        if match:
            url = match.group(0)
            url_found = True

            print(" VENTBUDDY IS LIVE!")
            print(f"\n Click here: {url}\n")
            print("✨ Your VentBuddy is ready!")
            break

    # Stop after 30 seconds if no URL found
    if not url_found and 'Ready' in line:
        time.sleep(2)

 Launching VentBuddy with Cloudflare Tunnel...

🧹 Cleaning up...
 Setting up Cloudflare Tunnel...
⏳ Starting Streamlit...
 Creating public tunnel (this takes ~20 seconds)...

2025-12-17T13:57:22Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-17T13:57:22Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-17T13:57:27Z INF +--------------------------------------------------------------------------------------------+
2025-12-17T13:57:27Z INF |